In [66]:
import os, glob
import json
import importlib
import radiometric_calib_utils
import mutils
import PIL.Image as Image
import extract_spectral
importlib.reload(radiometric_calib_utils)
importlib.reload(mutils)
importlib.reload(extract_spectral)
import mutils
import extract_spectral as espect
import matplotlib.pyplot as plt
import numpy as np
from math import ceil

# QAQC of labelled images

We will replot the bboxes on the images again using the info from `saved_bboxes`. The aim is to check if the drawn bboxes is drawn correctly on the images due to some bugs before the program was stable.

Previous bugs:

1. `saved_plots` were not saved together with `saved_bboxes` text files, which makes it hard to validate the bboxes drawn $\rightarrow$ this has been subsequently corrected by saving both the bboxes and plots. 

    Consequence: Some bboxes text files do not have a corresponding saved plots

2. Some bboxes from previous images were transferred over to subsequent images because the bboxes were not cleared ("reset") $\rightarrow$ this has been corrected

    Consequence: Some images had unwanted drawn bboxes which were wrong

3. Normalisation of RGB image $\rightarrow$ this has been subsequently corrected by providing true rgb images

    Consequence: Normalisation may have exaggerated the brightness of the image, resulting in possibly mislabelling regions as glint even though sun glint was not present



If data was transferred from another platform, the file path of the images saved in the text file will not be consistent with the file path in your local machine. We will need to reassign a new directory to the file path in the text file:

For example, your raw images was stored in the C drive, but mine is stored in the D drive, so when the program tries to search for images in my C drive, there is an error because my C drive doesn't have the raw images. This is why we need to reassign the main directory.

- directory (*this is the directory that needs to be reassigned*)
    - survey folder
        - flight folder
            - RawImg folder
                - ...raw images...

To reassign the images to new directory, simply specify where all the survey folders are stored in YOUR local machine e.g. `new_dir = r"D:\EPMC_flight"`

In [67]:
new_dir = r"D:\EPMC_flight"
ES = espect.VerifyBboxes(r"saved_bboxes", 
                         assign_new_dir=new_dir,split_iter=3)
store_bboxes = ES.store_bboxes()
if os.path.exists(list(store_bboxes)[0]):
    ES.plot_bboxes(show_n = None,figsize=(10,20),save=True)
else:
    print(f"QAQC plots not saved because {list(store_bboxes)[0]} does not exist!\nCheck if {new_dir} is the folder where survey images are stored")

Flights:  50%|█████     | 1/2 [00:11<00:11, 11.06s/it]Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Flights: 100%|██████████| 2/2 [00:40<00:00, 20.29s/it]


Else, if no reassignment is needed, just run the following:

In [60]:
ES = espect.VerifyBboxes(r"saved_bboxes")
store_bboxes = ES.store_bboxes()
if os.path.exists(list(store_bboxes)[0]):
    ES.plot_bboxes(show_n = None,figsize=(10,20),save=True)
else:
    print(f"QAQC plots not saved because {list(store_bboxes)[0]} does not exist!")

(919, 1239, 3)
(919, 1239, 3)


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


(919, 1239, 3)
